# Psuedocode biz question to data driven

In [ ]:
def retreive_abstract_to_data(es, tmg_abstract_index, embedder_model, question, reranker_model):
    index_name = tmg_abstract_index
    vector_field = 'embedding'
    question_encode = [list(i) for i in embedder_model.encode([question])]
    vector = question_encode[0]
    # print(vector)
    semantic_resp = search_vector(es,vector, index_name, vector_field)
    #semantic_resp
    list_of_abstract_question = []
    list_of_abstract_to_data_example = []
    list_of_table_description = []
    
    for hit in semantic_resp['hits']['hits']:
        list_of_abstract_to_data_example.append(hit['_source']['abstract_to_data_example'])
        list_of_table_description.append(hit['_source']['table_description'])
        list_of_abstract_question.append(hit['_source']['abstract_question'])

    question_list = []
    for abs_question in list_of_abstract_question:
        question_list.append([question, abs_question])
    
    th_scores = reranker_model.predict(question_list)
    abstract_to_data_example_prompt = list_of_abstract_to_data_example[th_scores.argmax()]
    table_description_prompt = list_of_table_description[th_scores.argmax()]
    return abstract_to_data_example_prompt, table_description_prompt

In [ ]:
# Abstract to Data driven
def create_data_driven_prompt(table_description, abstodata_example, question):
    template = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for data driven company question guideline <|eot_id|><|start_header_id|>user<|end_header_id|>

Here is the data dictionary I have below

Table Details:
{table_description}

Table Relations:
Table1:SALES_TRANSACTION.TRANSACTIONNUMBER [PK] to Table2: SALESTENDER_TRANSACTION.TRANSACTIONNUMBER [PK]
Table1:SALES_TRANSACTION.PLANTCODE [FK] to Table3: PLANTCODE.PLANTCODE [PK]
Table1:SALES_TRANSACTION.MATERIALCODE [FK] to Table4: PRODUCT_DIM.PRODUCTCODE [PK]
Table1:SALES_TRANSACTION.CUSTOMERCODE [FK] to Table5: MCARD_DIM.MCARDCODE [PK]
Table1:SALES_TRANSACTION.STORECODE [FK] to Table7: BRANCH_COMPANYCODE [PK]
Table5: MCARD_DIM.MEMBERID [FK] to Table6: DEMOGRAPHICS_DIM.MEMBERID [PK]

Can you give me examples of ABSTRACT question to DATA DRIVEN questions based on information in Table Details ans Table Relations? (ตอบเป็นภาษาไทย)
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{abstodata_example}
<|eot_id|><|start_header_id|>user<|end_header_id|>
OK your pattern is Good keep it this pattern.
Now I have my ABSTRACT question as below, please give me the DATA DRIVEN question example.
(ตอบเป็นภาษาไทย)
ABSTRACT:
{question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
DATA DRIVEN:
"""
    return template


In [ ]:
def send_to_watsonxai(model,
                    prompts
                    ):
    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "make sure none of the prompts in the inputs prompts are empty"
    output = []
    for prompt in prompts:
        o = model.generate_text(prompt)
        output.append(o)
    return output

In [ ]:
question = "ทำไมสาขา 17 ยอดตกลงเยอะจัง"

abstract_to_data_example_prompt, table_description_prompt = retreive_abstract_to_data(es,tmg_abstract_index,embedder_model, question, reranker_model)

prompt_scope = create_data_driven_prompt(table_description_prompt, abstract_to_data_example_prompt, question)

o = send_to_watsonxai(abstract_to_data_llm_model, [prompt_scope])

data_driven_question_list = ast.literal_eval(o[0])

real_list = data_driven_question_list

results = {"result": real_list}

# Psuedocode data driven to sql

In [ ]:
def retreive_dataq_to_sql(es, tmg_sql_index, embedder_model, chosen_question, reranker_model):
    index_name = tmg_sql_index
    vector_field = 'embedding'  
    query_encode = [list(i) for i in embedder_model.encode([chosen_question])]
    vector = query_encode[0]
    print(len(vector))
    # print(vector)
    semantic_resp = search_vector(es, vector, index_name, vector_field)
    #semantic_resp
    list_of_ask_question = []
    list_of_question_example = []
    list_of_table_description = []
    # print(semantic_resp)
    print(semantic_resp['hits']['hits'][0]['_source'].keys())
    for hit in semantic_resp['hits']['hits']:
        list_of_ask_question.append(hit['_source']['ask_question'])
        list_of_question_example.append(hit['_source']['question_example'])
        list_of_table_description.append(hit['_source']['table_description'])

    question_list = []
    for ask_question in list_of_ask_question:
        question_list.append([chosen_question, ask_question])
    
    th_scores = reranker_model.predict(question_list)        
    question_example_prompt = list_of_question_example[th_scores.argmax()]
    table_description_prompt = list_of_table_description[th_scores.argmax()]
    return question_example_prompt, table_description_prompt

In [ ]:
def sql_generation_prompt(table_description, example, question):
    prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant for data driven question to sql generation.
Use data only in the Table Descriptions section that user gave you.
If do not have data please says: ข้อมูลไม่เพียงพอ!
<|eot_id|><|start_header_id|>user<|end_header_id|>
Here is the data dictionary I have below

Table Details:

{table_description}

Table Relations:
Table1:SALES_TRANSACTION.TRANSACTIONNUMBER [PK] to Table2: SALESTENDER_TRANSACTION.TRANSACTIONNUMBER [PK]
Table1:SALES_TRANSACTION.PLANTCODE [FK] to Table3: PLANTCODE.PLANTCODE [PK]
Table1:SALES_TRANSACTION.MATERIALCODE [FK] to Table4: PRODUCT_DIM.PRODUCTCODE [PK]
Table1:SALES_TRANSACTION.CUSTOMERCODE [FK] to Table5: MCARD_DIM.MCARDCODE [PK]
Table1:SALES_TRANSACTION.STORECODE [FK] to Table7: BRANCH_COMPANYCODE [PK]
Table5: MCARD_DIM.MEMBERID [FK] to Table6: DEMOGRAPHICS_DIM.MEMBERID [PK]

I use netezza as my SQL generator.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Thank you! This is a good data. How can I help you?
<|eot_id|><|start_header_id|>user<|end_header_id|>
Below are examples of my company's queries.
{example}

Question:
    อะไรคือการนอนหลับ

SQL:
SQL unable to generate.

Give me the sql script of my question.
Use only data existing in Table Details.
If it is too complex please return 'SQL unable to generate'
---

Question:
    {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SQL:
    """
    return prompt

In [ ]:
chosen_question = question_to_sql

question_example_prompt, table_description_prompt = retreive_dataq_to_sql(es, tmg_sql_index, embedder_model, chosen_question, reranker_model)

prompt_sql = sql_generation_prompt(table_description_prompt, question_example_prompt, chosen_question)

o = send_to_watsonxai(sql_gen_llm_model, [prompt_sql])

sql_script = o[0]

sql_script = sql_script.replace('\\', '')

file_path = 'temp/sql_script.txt'  
os.makedirs(os.path.dirname(file_path), exist_ok=True)
if os.path.exists(file_path):
    os.remove(file_path)

with open(file_path, 'w') as file:
    file.write(sql_script)

results = {"results": sql_script}